#  Actionner un servo-moteur  standard

Les servo-moteurs sont des organes essentiels des robots ce sont eux qui fournissent les mouvements élémentaires qui combinés entre eux peuvent réaliser des mouvements complexes. Il y a trois types de moteurs :
 * Les moteurs à courant continu qui permettent de mettre en oeuvre des puissances et des vitesses importantes mais contrôlées, ils peuvent fonctionner dans les deux sens et ne sont pas limités dans l'amplitude du mouvement, ils sont connectés par deux fils reliés à une puce spécialisée qui gère l'intensité et le sens du courant envoyé;
 * Les servo moteurs comme ceux que nous allons utiliser qui fonctionnent avec de basses tensions, ils ont une amplitude de mouvement limitée 90° ou 180° en général, ils sont connectés avec 3 fils, le + le - et un fil de contrôle fournissant l'angle de rotation, leur vitesse n'est pas contrôlée
 * Les moteurs pas à pas qui permettent de réaliser des mouvements très précis car leur rotation se fait par des pas angulaires élémentaires très petits, ils sont connectés pa 4 fils.
 
Nous allons utiliser le bras robotisé MeArm équipé de 4 servo moteurs.
 
 ## Connexion  du premier servo-moteur 
 
 Nous utilisons d'abord le servo-moteur de rotation du bras dans le plan horizontal connecté à la broche N° 33 (GPIO #13) et la bibliothèque RPi.GPIO. Les servo-moteurs se contrôlent à une fréquence de 50 Hz soit une période (`Period`) de 0.02 secondes correspondant à 20 millisecondesla configuration de la Raspberry se fait donc avec la commande `pwm1 = gpio.PWM(33, 50)`  qui donne le nom pwm1 au dispositif de contrôle de la broche 33 à 50 Hz.
 L'angle de rotation est contrôlé par la durée en millisecondes de l'impulsion (`Pulsewidth`) envoyée par le dispositif de contrôle :
 
 * Pour un servo moteur avec 90° de liberté
     * 1 milliseconde position extrême gauche
     * 1.5 milliseconde position intermédiaire
     * 2 milliseconde à l'extrême droite
 * Pour un servo moteur avec 180° de liberté
     * 0.5 milliseconde position extrême gauche
     * 1.5 milliseconde position intermédiaire
     * 2.5 milliseconde à l'extrême droite
 
La commande permettant de modifier la durée de l'impulsion est ChangeDutyCycle() mais elle n'agit pas directement sur l'impulsion mais sur la charge (DutyCycle) exprimée en pourcent (%) qui sont reliés par la relation :  
 `DutyCycle = PulseWidth/Period`. 
 
 * Pour avoir une impulsion de 1 milliseconde  on doit fournir la charge :
     * DutyCycle = 0.001 /  0.02 = 0.05 = 5%
 * Pour avoir une impulsion de 1.5 milliseconde  on doit fournir la charge :
     * DutyCycle = 0.0015 /  0.02 = 0.075 = 7.5%
 * Pour avoir une impulsion de 2 milliseconde  on doit fournir la charge :
     * DutyCycle = 0.002 /  0.02 = 0.1 = 10%
 * Pour avoir une impulsion de 0.5 milliseconde  on doit fournir la charge :
     * DutyCycle = 0.0005 /  0.02 = 0.025 = 2.5%
 * Pour avoir une impulsion de 2.5 milliseconde  on doit fournir la charge :
     * DutyCycle = 0.0025 /  0.02 = 0.125 = 12.5%
     
Le programme suivant démarre en position centrale puis effectue une rotation de 90° vers la droite (moteur inversé)  puis une rotation de 180° vers la gauche et retour à la position neutre. les deux premières lignes ne sont exécutées qu'une fois.

In [114]:
import mraa, time, upm,RPi.GPIO as gpio
gpio.setmode(gpio.BOARD)  # BCM ou BOARD pour identifier les broches. choisir BOARD

In [124]:
gpio.setup(33, gpio.OUT)
pwm1 = gpio.PWM(33, 50)
pwm1.start(7.5)
time.sleep(1)
pwm1.ChangeDutyCycle(2.5)
time.sleep(1)
pwm1.ChangeDutyCycle(12.5)
time.sleep(1)
pwm1.ChangeDutyCycle(7.5)
time.sleep(1)
pwm1.stop()

## Connexion de tous les servo-moteurs en position neutre

Les 4 servo-moteurs sont allumés en position neutre, le bras est trop en avant et n'est plus contrôlé. Les servo-moteurs seront utilisés sur une partie seulement de la plage disponible.

In [45]:
gpio.setup(33, gpio.OUT) # Rotation dans le plan horizontal
gpio.setup(32, gpio.OUT) # Mouvement avant arrière du bras
gpio.setup(36, gpio.OUT) # Mouvement haut bas du bras
gpio.setup(35, gpio.OUT) # Mouvement de pince
pwm1 = gpio.PWM(33, 50)
pwm2 = gpio.PWM(32, 50)
pwm3 = gpio.PWM(36, 50)
pwm4 = gpio.PWM(35, 50)
pwm1.start(7.5)
pwm2.start(7.5)
pwm3.start(7.5)
pwm4.start(7.5)
time.sleep(1)
pwm1.stop()
pwm2.stop()
pwm3.stop()
pwm4.stop()

##  Mouvement avant du bras 

Le bras démarre en position verticale s'avance et revient en position verticale

In [125]:
gpio.setup(32, gpio.OUT) # Mouvement avant arrière du bras
pwm2 = gpio.PWM(32, 50)
pwm2.start(2)
time.sleep(1)
pwm2.ChangeDutyCycle(4)
time.sleep(1)
pwm2.ChangeDutyCycle(2)
time.sleep(1)
pwm2.stop()

## Mouvement Haut bas  du bras

La puissance des servo-moteurs est insuffisante pour remonter le bras s'il est trop avancé, on démarre en position semi élevée, le bras monte puis redescend. Pour aider au mouvement le bras avance puis recule.

In [126]:
gpio.setup(36, gpio.OUT) # Mouvement haut bas du bras
gpio.setup(32, gpio.OUT) # Mouvement avant arrière du bras
pwm2 = gpio.PWM(32, 50)
pwm3 = gpio.PWM(36, 50)
pwm2.start(4)
pwm3.start(10)
time.sleep(1)
pwm3.ChangeDutyCycle(10)
time.sleep(1)
pwm3.ChangeDutyCycle(12.5)
time.sleep(1)
pwm2.ChangeDutyCycle(2)
time.sleep(1)
pwm3.ChangeDutyCycle(9)
time.sleep(1)
pwm2.stop()
pwm3.stop()

## Mouvement de la pince

La pince démarre ouverte et est fermée complètement ensuite.

In [122]:
gpio.setup(35, gpio.OUT) # Mouvement de pince
pwm4 = gpio.PWM(35, 50)
pwm4.start(1.5)
time.sleep(1)
pwm4.ChangeDutyCycle(6)
time.sleep(0.5)
pwm4.stop()

Remise à zéro de toutes les broches

In [113]:
gpio.cleanup() 

# Exercice

En utilisant les éléments de programmatin ci-dessus réaliser un mouvement de prise d'un élément d'un côté du bras et dépôt en hauteur de l'autre côté.